In [2]:
#data load
from __future__ import print_function
import os
import pandas as pd
import numpy as np


data_path = [ 'data']
filepath = os.sep.join(data_path + ['train.csv'])
data = pd.read_csv(filepath, sep=',', header=0)
data.dtypes

Name                  object
Location              object
Year                   int64
Kilometers_Driven      int64
Fuel_Type             object
Transmission          object
Owner_Type            object
Mileage               object
Engine                object
Power                 object
Seats                float64
New_Price             object
Price                float64
dtype: object

In [3]:
#NaN행 제거

# data = data[data['Mileage'].notna()]
# print(data.shape)
# data = data[data['Engine'].notna()]
# print(data.shape)
# data = data[data['Power'].notna()]
# print(data.shape)
# data = data[data['Seats'].notna()]
# print(data.shape)
import seaborn as sns

data.drop(labels="New_Price", axis = 1,inplace=True)
data = data.dropna()

In [4]:
#target 분리
X = data.drop(labels="Price", axis = 1)
y = data.Price.astype(int)
print(X.shape)
print(y.shape)

(5975, 11)
(5975,)


In [5]:
#단위제거
X = X.reset_index(drop=True)
for i in range(X.shape[0]):
    X.at[i, 'Company'] = X['Name'][i].split()[0]
    X.at[i, 'Mileage(km/kg)'] = X['Mileage'][i].split()[0]
    X.at[i, 'Engine(CC)'] = X['Engine'][i].split()[0]
    X.at[i, 'Power(bhp)'] = X['Power'][i].split()[0]

In [6]:
#불필요한 column drop

X.drop(["Name"],axis=1,inplace=True)
X.drop(["Mileage"],axis=1,inplace=True)
X.drop(["Engine"],axis=1,inplace=True)
X.drop(["Power"],axis=1,inplace=True)

In [7]:
X.isnull().sum()
# X.head()

Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Seats                0
Company              0
Mileage(km/kg)       0
Engine(CC)           0
Power(bhp)           0
dtype: int64

In [8]:
#data 인코딩
from sklearn.preprocessing import LabelEncoder
from IPython.display import display


number = LabelEncoder()


X['Location'] = number.fit_transform(X['Location']).astype(int)
X['Fuel_Type'] = number.fit_transform(X['Fuel_Type']).astype(int)
X['Transmission'] = number.fit_transform(X['Transmission']).astype(int)
X['Company'] = number.fit_transform(X['Company']).astype(int)
X['Owner_Type'] = number.fit_transform(X['Owner_Type']).astype(int)
X['Power(bhp)'] = number.fit_transform(X['Power(bhp)']).astype(int)


X.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Seats,Company,Mileage(km/kg),Engine(CC),Power(bhp)
0,9,2010,72000,0,1,0,5.0,18,26.6,998,259
1,10,2015,41000,1,1,0,5.0,10,19.67,1582,56
2,2,2011,46000,3,1,0,5.0,9,18.2,1199,343
3,2,2012,87000,1,1,0,7.0,18,20.77,1248,345
4,3,2013,40670,1,0,2,5.0,1,15.2,1968,76


In [9]:
#data type int로 변경
one_hot_encode_cols = X.dtypes[X.dtypes != float]  # filtering by string categoricals
one_hot_encode_cols = one_hot_encode_cols.index.tolist()  # list of categorical fields

for col in one_hot_encode_cols:
    X[col] = X[col].astype(float)
one_hot_encode_cols = X.dtypes[X.dtypes != int]  # filtering by string categoricals
one_hot_encode_cols = one_hot_encode_cols.index.tolist()  # list of categorical fields

for col in one_hot_encode_cols:
    X[col] = X[col].astype(int)

X.dtypes

Location             int32
Year                 int32
Kilometers_Driven    int32
Fuel_Type            int32
Transmission         int32
Owner_Type           int32
Seats                int32
Company              int32
Mileage(km/kg)       int32
Engine(CC)           int32
Power(bhp)           int32
dtype: object

In [10]:
#skew 확인
skew = pd.DataFrame(X.skew())
skew.columns = ['skew']
skew['too_skewed'] = skew['skew'] > .95
skew

,skew,too_skewed
Location,-0.132254,False
Year,-0.841781,False
Kilometers_Driven,58.598903,True
Fuel_Type,0.144601,False
Transmission,-0.947236,False
Owner_Type,1.798632,True
Seats,1.835262,True
Company,0.107527,False
Mileage(km/kg),-0.304901,False
Engine(CC),1.425444,True


In [11]:
#최솟값 확인 및 더하기
X.describe()
X['Owner_Type']=X['Owner_Type']+1
X['Seats']=X['Seats']+1

In [12]:
#ox-Cox Transform로 skew 낮추기
from pandas import Series
from scipy import stats

X["Kilometers_Driven"] = pd.Series(stats.boxcox(X["Kilometers_Driven"])[0])
X["Engine(CC)"] = pd.Series(stats.boxcox(X["Engine(CC)"])[0])
X["Seats"] = pd.Series(stats.boxcox(X["Seats"])[0])
X["Owner_Type"] = pd.Series(stats.boxcox(X["Owner_Type"])[0])


skew = pd.DataFrame(X.skew())
skew.columns = ['skew']
skew['too_skewed'] = skew['skew'] > .95
skew


,skew,too_skewed
Location,-0.132254,False
Year,-0.841781,False
Kilometers_Driven,0.328568,False
Fuel_Type,0.144601,False
Transmission,-0.947236,False
Owner_Type,1.671487,True
Seats,-0.137029,False
Company,0.107527,False
Mileage(km/kg),-0.304901,False
Engine(CC),0.007785,False


In [13]:
#데이터 분류
from sklearn.model_selection import train_test_split
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
#NaiveBayes 적용
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
    
model = GaussianNB()
pred = model.fit(X_train,y_train).predict(x_test)

In [15]:
#RMSE
from sklearn.metrics import mean_squared_error


print("Rmse : ", mean_squared_error(y_test,pred)**0.5)

Rmse :  11.488761209666533


## NaiveBayes 결과

### 순서

1. 데이터 로드 후 Nan data가 많은 'New Price' column drop 후 Nan data가 포함된 data가 있는 행을 제거

2. 데이터 전처리
    1) 단위가 붙은 column의 단위를 제거
    2) 불필요한 column drop
    3) 숫자가 아닌 column들을 int로 인코딩
    4) 모든 data type을 int로 변경

3. skew()계산 후 너무 왜도가 심하면 log를 취하여 해결

4. Naive Bayes 적용
   - 3가지 방법중 가장 RMSE가 낮은 GaussianNB방법 채택

5. RMSE 계산
   - 결과 : 11.488761209666533

##### Naive Bayes는 분류에 최적화 되어있고 '기본적으로 모든 column이 독립적이라는 전제를 갖는다. 그러나 이 경우에서는 모든 column이 독립적이지 못하고 모든 숫자들을 int형로 변환해야하기 때문에 정확도가 낮게 나온 것으로 추측된다.
   
### 결과
1. KNN
   RMSE : 5.768876430907574
   
   - 단순한 KNN의 특성상 K의 값만 잘 설정하면 좋은 성능을 낼 수 있다.
   - 또한 이 data의 경우 데이터의 양이 많지 않기 때문에 적당히 좋은 결과를 낸 것으로 추측된다. 
   
2. NaiveBayes 
   RMSE : 11.488761209666533
   
   - Naive Bayes는 분류에 최적화 되어있고 '기본적으로 모든 column이 독립적이라는 전제를 갖는다. 그러나 data의 특성 상 모든 column이 독립적이지 못한다.
   - 또한 모든 숫자들을 int형로 변환해야하기 때문에 정확도가 낮게 나온 것으로 추측된다.
   
3. SVM 
   RMSE : 1044.9557155198318
   
   - Naive Bayes와 마찬가지로 float형을 모두 int형으로 변환했기 때문에 이에 따른 오차가 발생한 것으로 추측된다.
   - 따라서 target data가 float형인 해당 data와 적합하지 않은 듯하다.
   
4. Decision Tree 
   RMSE : 3.7793213100801744
   
   - DT는 기본적으로 분류와 회귀 모두 가능하다. 따라서 범주나 연속형 수치 모두 예측할 수 있다.
   - 

